In [1]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage,ToolMessage,AIMessageChunk
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain.schema.runnable import Runnable,RunnableLambda,RunnableSequence,RunnablePassthrough,RunnableParallel

In [2]:
load_dotenv()
# Access them
endpoint = os.getenv("LANGCHAIN_ENDPOINT")
api_key = os.getenv("LANGCHAIN_API_KEY")
open_ai_api_key=os.getenv("open_ai_api_key")
# print("LangChain Endpoint:", endpoint)
# print("API Key:", api_key)

In [3]:
model=ChatOpenAI(
    temperature=0.7,
    model_name="gpt-4-turbo",
    api_key=open_ai_api_key,
    max_tokens=50
)

In [4]:
Prompt_Template = ChatPromptTemplate.from_messages([
        ("system","You are an expert product reviewer."),
        ("human","List the main features of the product {product_name}."),
    ])

In [5]:
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages([
        ("system","You are an expert product reviewer."),
        ("human","Given these features: {features},list pros of the features.",),
    ]
    )
    return pros_template.format_prompt(features=features)

In [6]:
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages([
        ("system","You are an expert product reviewer."),
        ("human","Given these features: {features},list cons of the features.",),
    ]
    )
    return cons_template.format_prompt(features=features)

In [7]:
def combine_pros_cons(pros,cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"

In [8]:
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)


In [9]:
chain=(
    Prompt_Template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros":pros_branch_chain,"cons":cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"],x["branches"]["cons"]))
)

In [10]:
result =chain.invoke({"product_name": "MacBook Pro"})

In [12]:
print(result)

Pros:
The MacBook Pro, as updated in December 2023, is designed to meet the needs of professionals and creatives with several high-performance features. Below are some of the key features generally found in MacBook Pro models around that time, along with the advantages

Cons:
Certainly! While the MacBook Pro is renowned for its high-performance capabilities and sleek design, every feature has potential downsides depending on user needs and preferences. Below, I'll outline some cons related to typical features found in MacBook Pro models as of late 
